# Match most similar competency

## Existing competencies

In [11]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [12]:
competencys = ['Creativity', 'Employee loyalty', 'Reliability', 'Leadership', 'Passion', 'People Skills', 'Proactivity', 'Team Work']
for i in range(8):
    print "{}:{}".format(i, competencys[i])


0:Creativity
1:Employee loyalty
2:Reliability
3:Leadership
4:Passion
5:People Skills
6:Proactivity
7:Team Work


In [13]:
descriptions = ['Ability to think in a divergent way, and generate novel ideas.',
                'Be committed to the success of the organization and believe that working for this organization is their best option.',
                'Ability to performing to perform when needed, finishing projects and meeting deadlines.', 
                'Ability to engage and facilitate others to create a better result.', 
                'Desire to provide long-term commitment to their organization, demonstrate peak performance, and maintain increased tenure with the organization. With the intense desire or enthusiasm for the work.', 
                'Ability to establish social connections and understand the mindset of others.', 
                'Take the initiative and responsibility in improving business, rather than looking for causes in outside circumstances or other people.',
                'Ability to adapt to the needs of a group of people, while supporting the work of others.']

## load dictionary and corpus

In [14]:
from gensim import corpora, similarities, models

dictionary = corpora.Dictionary.load_from_text('/mnt/local/var/seedlink/data/dictionary/dict_en.txt')
print(dictionary)

Dictionary(50920 unique tokens: [u'sucess', u"cake'", u'mid-week', u'stallions', u'sonja']...)


In [15]:
import codecs
import json
path = '/mnt/local/var/seedlink/data/nltk_data/corpora/prepcorp/en_cc.json'
with codecs.open(path, 'r', 'utf8') as fp:
    jdata = fp.read()
data = json.loads(jdata)
corpus = data

In [16]:
for i in range(10):
    words =  [dictionary[t[0]] for t in corpus[i]]
     #print corpus[i]
    #print ' '.join(words)

## New competency

In [17]:
new_comp = """
Charisma	Able to establish social connections and a broad network
Confidence	Acts on his/her own initiative with confidence
Creativity	Seeks change and performs creatively
Entrepreneurial	Keeps aware of both organizational issues and market opportunities
Goal Orientated	Works systematically and drives the project to its goals
Goal Orientation	Work systematically and drive the project to the target goals
Growth Potential	Growth Potential
Integrity	Acts with integrity and shows social responsibility
Intellect	Flexible application of knowledge to differing contexts
Job Performance	Job Performance
Leadership	Identifies talents, empowers and motivates other
Logic	Logical thinking and rational analysis
Organisation	Sets objectives, balances resources and time, monitors progress
Organization	Sets objectives, balances resources and time, monitors progress
Performance	Job Performance
Proactive	Proactively deals with ambiguity
Process driven	Demonstrate commitment, accountablity, follow policies and procedures of the organizations
Resilient	Coping and resilient under stressful environment
Self motivated	Work enthusiastically for the achievement of self-development
Strategic Competency	Strategic Competency
Strategy	Foresight for potential problems and comes up with appropriate solutions
Teamwork	Adapts to the team; Supports others
"""

lines = new_comp.lower().strip().split('\n')
new_comp_dict = {splitted[0]:splitted[1] for splitted in [line.split('\t') for line in lines]}
new_comp_dict

{'charisma': 'able to establish social connections and a broad network',
 'confidence': 'acts on his/her own initiative with confidence',
 'creativity': 'seeks change and performs creatively',
 'entrepreneurial': 'keeps aware of both organizational issues and market opportunities',
 'goal orientated': 'works systematically and drives the project to its goals',
 'goal orientation': 'work systematically and drive the project to the target goals',
 'growth potential': 'growth potential',
 'integrity': 'acts with integrity and shows social responsibility',
 'intellect': 'flexible application of knowledge to differing contexts',
 'job performance': 'job performance',
 'leadership': 'identifies talents, empowers and motivates other',
 'logic': 'logical thinking and rational analysis',
 'organisation': 'sets objectives, balances resources and time, monitors progress',
 'organization': 'sets objectives, balances resources and time, monitors progress',
 'performance': 'job performance',
 'proac

## Tfidf

In [18]:
tfidf = models.TfidfModel(corpus)

In [19]:
desc_bow = [dictionary.doc2bow(desc.lower().split()) for desc in descriptions]
desc_tfidf = tfidf[desc_bow]

## LDA

In [20]:
lda = models.LdaModel(tfidf[corpus], num_topics=100, id2word=dictionary)

In [21]:
lda.save('/tmp/competency-100-orig.lda')

In [ ]:
lda.update(desc_tfidf)

In [22]:
lda.print_topics(5)

[(76,
  u'0.036*"turn" + 0.022*"may" + 0.022*"sequel" + 0.022*"direction" + 0.022*"ends" + 0.021*"cover" + 0.021*"purpose" + 0.020*"look" + 0.019*"immediately" + 0.018*"message"'),
 (82,
  u'0.020*"rather" + 0.017*"house" + 0.016*"big" + 0.014*"somewhat" + 0.012*"fascinating" + 0.011*"imagination" + 0.011*"screen" + 0.010*"tried" + 0.009*"images" + 0.009*"proceedings"'),
 (42,
  u'0.045*"don" + 0.031*"simply" + 0.030*"really" + 0.027*"we" + 0.025*"wanted" + 0.023*"daughter" + 0.022*"know" + 0.022*"feel" + 0.022*"t" + 0.022*"meet"'),
 (40,
  u'0.047*"funny" + 0.025*"jackie" + 0.023*"present" + 0.018*"thrown" + 0.017*"forgotten" + 0.016*"throw" + 0.016*"directors" + 0.016*"playing" + 0.015*"frank" + 0.015*"female"'),
 (72,
  u'0.018*"five" + 0.017*"gone" + 0.017*"solid" + 0.015*"nora" + 0.013*"sick" + 0.013*"self" + 0.012*"powers" + 0.012*"partner" + 0.011*"mayor" + 0.011*"intended"')]

In [25]:
index = similarities.MatrixSimilarity(lda[desc_tfidf]) # lda model

In [26]:
query = 'Able to establish social connections and a broad network'.lower()
query_bow = dictionary.doc2bow(query.split())
print query_bow
query_lda = lda[tfidf[query_bow]]
query_lda
sims = index[query_lda]


[(3, 1), (4, 1), (14, 1), (1588, 1), (3670, 1), (6227, 1), (9190, 1), (12257, 1), (15255, 1)]


In [27]:
competencys[3]

'Leadership'

In [30]:
# query by competency name
for comp in new_comp_dict:
    query = comp.lower()
    query_bow = dictionary.doc2bow(query.split())
    query_lda = lda[tfidf[query_bow]]

    sims = index[query_lda]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency     	Passion             	0.173278793693
growth potential         	Reliability         	0.353087931871
integrity                	Creativity          	0.0
intellect                	Proactivity         	0.144906938076
confidence               	Creativity          	0.0
goal orientated          	Employee loyalty    	0.415975034237
job performance          	Creativity          	0.0
organisation             	Creativity          	0.752248764038
creativity               	Creativity          	0.0
self motivated           	Proactivity         	0.681162118912
strategy                 	Creativity          	0.0
charisma                 	Passion             	0.300692051649
teamwork                 	Creativity          	0.0
resilient                	Creativity          	0.0
performance              	Creativity          	0.0
goal orientation         	Creativity          	0.40226662159
leadership               	Creativity          	0.0
entrepreneurial          	Creativity          

In [31]:
# query by competency desc
for comp in new_comp_dict:
    query = new_comp_dict[comp].lower()
    query_bow = dictionary.doc2bow(query.split())
    query_lda = lda[tfidf[query_bow]]

    sims = index[query_lda]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency     	Passion             	0.173278793693
growth potential         	Reliability         	0.353087931871
integrity                	Proactivity         	0.25954964757
intellect                	Reliability         	0.20600028336
confidence               	Proactivity         	0.375999569893
goal orientated          	Proactivity         	0.153876557946
job performance          	Creativity          	0.0
organisation             	Proactivity         	0.5038869977
creativity               	Passion             	0.0984498485923
self motivated           	Reliability         	0.228799894452
strategy                 	Employee loyalty    	0.339382588863
charisma                 	People Skills       	0.514912486076
teamwork                 	Leadership          	0.242369458079
resilient                	Passion             	0.127446353436
performance              	Creativity          	0.0
goal orientation         	Proactivity         	0.225374698639
leadership               	Proacti